# Quantum Proof of Work with Parametrized Quantum Circuits
#### Mikhail Shalaginov (mys@mit.edu) and Michael Dubrovsky (mike@powx.org)

<b>Abstract</b>:
Despite all the progress in quantum technologies over the last decade, there is still a dearth of practical applications for quantum computers with a small number of noisy qubits. The effort to show quantum supremacy has been largely focused on demonstrating computations that cannot be accomplished on a classical computer at all, a difficult and controversial target. Quantum advantage (a speedup over classical computers) is a more practical milestone for today's modest quantum processors. In this work, we proposed a scheme for quantum-computer compatible proof of work (cryptographic mechanism used in Bitcoin mining) and verified it on a n-qubit quantum processor.

<br>
In this demo we demonstrate the construction of a block-chain composed of five consequtive blocks (input parameter 'numberOfBlocks') by running the qPoW algorithm on 4-qubits ('n_qreg = 4') of a selected IBM Q node (e.g., 'qpu_name = <a href="https://quantum-computing.ibm.com/">ibmq_quito</a>').

<b>Schematic of the qPoW cycle</b>:

<img src="images/schematic1.png" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="850 px" align="center">

<li>The input block data aka 'text' (e.g., <font color='green'>'4Schroedinger paid Einstein 1 qBTC04ca1a782621a440d03b5d87ecff8b68e2cc6124f57957b49a76bca91dede3a81'</font>) is a concatenated string containing: 1) current block # (<font color='green'>'4'</font>), 2) transaction information (<font color='green'>'Schroedinger paid Einstein 1 qBTC'</font>), 3) hash-string from a previous block (<font color='green'>'qBTC04ca1a782621a440d03b5d87ecff8b68e2cc6124f57957b49a76bca91dede3a'</font>) , and 4) a nonce (<font color='green'>'81'</font>). A nonce is a 32-bit integer randomly generated for each block mining cycle.</li> 
<br>
<li>By applying the first SHA3 operation the 'text' is transformed into a 256-bit hash-string (e.g., <font color='green'>'e1e5575da3a9e86da135552facddcc1ff44dd26502d0bc2b22961383f8b187ca'</font>). Noteworhy, it is almost impossible to predict the output of the SHA function, even a change of a single character will change the output string completely.</li> 
<br>
<li>Then the binary string is encoded into the parameters of the quantum circuit ansatz (detailed further). The quantum circuit produces a distribution of all the possible quantum states, the maximum (most probable) quantum state (e.g. <font color='green'>'0010'</font> ) is selected.</li> 
<br>
<li>We concatenate the maximum state with the string of zeros, so that the total lenght of the string becomes 256 bits.</li> 
<br>
<li>Then the XOR operation is performed on the maximum state 256-bit string and the binary string of the first hashing.</li> 
<br>
<li>The resulting binary string is hashed again.</li> 
<br>
<li>The output string (<font color='green'>'f307b3db12a649563831e3e1328c3c7a5b15ee541afaab563727cb992cf9d1ca'</font>) has to pass the difficulty check. If the output satisfies the conditions set by the difficulty threshold, i.e. the output string starts with a certain number of zeros (for the lowest difficulty - the string should start from at least one zero), then the block is reported to be successfully found. Otherwise, the output string is discarded and the next mina new nonce is randomly generated for the next cycle (red dashed line).</li> 
<br>
<li>After the block is finally found (i.e., the difficulty check is passed), the found nonce value has to be varified by everyone else in the network. The verification step is calculated with a qasm simulator running on a classical computer.</li>
<br>

<b>Drawing illustrating the task of quantum sampling performed by a pseudo-random parametrized quantum circuit</b>:

<img src="images/schematic2.png" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="750 px" align="center"></img>

<li>Pre-hashed 256-bit string (after the first SHA3) is split into 4-bit subgroups.</li> 
<br>
<li>Each 4-bit string is translated into a decimal number which is then introduced into one of the rotational gates as multiplier of pi/8 (angular unit).</li>
<br>
<li>The quantum gates in this specific ansatz are Rx, Rz and cRx.</li>
<br>
<li>Each gate and 4-bit stri are assigned a corresponding index number.</li>
<br>
<li>If the number of gates is more than 64 (the number of 4-bit subgroups), then the subgroups can be reused multiple times, as many times as needed.</li> 
<br>
<li>At the end, each qubit is measurd, which overall yields an n-dimensional qubit state.</li>
<br>
<li>From the quantum state distribution we select one state with the maximum probability.</li>
<br>
<li>The quantum sampling experiments for both quantum processor and qasm simulator are repeated 20,000 (maximum # of iterations on IBM Q nodes) to get accurate and reproducible results.</li>
<br>    
<li>For running the sampling task on a quantum node the ansatz should be transpiled, translated into native gates and adapted to the topology of the physical qubits.</li>
<br>

In [ ]:
# import modules for SHA
import sys
import hashlib
import numpy as np
import random

if sys.version_info < (3, 6):
	import sha3

# import modules for Qiskit
import qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import BasicAer, Aer, IBMQ, execute, schedule
from qiskit.compiler import transpile
from qiskit.tools.visualization import plot_histogram
from qiskit.transpiler import PassManager
from qiskit.tools.monitor import job_monitor
from qiskit.providers.aer import AerSimulator
# from azure.quantum.qiskit import AzureQuantumProvider

# import other useful packages 
from numpy import pi
from datetime import date
import time

# settings for IBM Q backends
# TOKEN = ''
# IBMQ.save_account(TOKEN)
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
qpu_name = 'ibmq_quito'
simulator_name = 'qasm_simulator'
qpu_backend = provider.get_backend(qpu_name)

# create data files
today = date.today()
date = today.strftime("%Y%m%d")
filepath = '' #specify data storage folder
filename_results = filepath + date + '_results_IBM_qpu.txt'
filename_log = filepath + date + '_log_IBM5_qpu.txt'
filename_bc = filepath + date + '_block_chain_qpu.txt'

# input parameters
previous_hash = '' #empty hash
block_number = 0
numberOfBlocks = 1
errors = 0 #number of 'False' nonces
difficulty = 1 #number of prefix zeros (hash-problem difficulty)
transactions ='Schroedinger paid Einstein 1 qBTC'
MAX_NONCE = 2**32
QC_switch = 1 #0 - simulator, 1 - real QC
n_qreg = 3 #number of qubits

# loop for building a blockchain
counter = 0
while counter < numberOfBlocks:

    with open(filename_log, 'a') as o:
        o.write('mining started \n')
        o.write('difficulty:'+ str(difficulty)+ '\n')
        o.write('previous_hash:' + str(previous_hash)+ '\n')
        o.write('transactions:' + str(transactions) +'\n')
        o.write('block_number:' + str(block_number) + '\n')
        o.write('QC_switch:' + str(QC_switch) + '\n\n')

    with open(filename_results, 'a') as o:
        o.write('mining started \n')
        o.write('difficulty:'+ str(difficulty)+ '\n')
        o.write('previous_hash:' + str(previous_hash)+ '\n')
        o.write('transactions:' + str(transactions) +'\n')
        o.write('block_number:' + str(block_number) + '\n')
        o.write('QC_switch:' + str(QC_switch) + '\n\n')

    if QC_switch == 1:
        with open(filename_results, 'a') as o:
            o.write('IBM node:'+ qpu_name)
    else:
        with open(filename_results, 'a') as o:
            o.write('IBM node:'+ simulator_name)

### Mining functions

    # QPoW cycle: function takes the input text (block number, nonce,...) and pushes it through once 
    def qPoW(text, QC_switch, quantum_circuit, nonce):

        hashIn = hashlib.sha3_256(text.encode("ascii")).hexdigest() # hashing the 'text' input
        #string-type output
        
        print ('hashIn-hex:', hashIn, 'length:', len(hashIn))
        with open(filename_log, 'a') as o:
            o.write('hashIn-hex:' + str(hashIn)+ '\n')

        # convert hashIn(hex) to hashIn_bin(binary)
        scale = 16 #hex base
        hashIn_bin = bin(int(hashIn, scale))[2:].zfill(len(hashIn)*4)
        print ('hashIn-binary:', str(hashIn_bin), 'length:', len(hashIn_bin))
        with open(filename_log, 'a') as o:
            o.write('hashIn-binary:' + str(hashIn_bin)+ '\n')

        #switch to enable quantum simulator or quantum computer
        if QC_switch == 0:
            [qstate_bin, comp_time] = sim_quantum_operation(quantum_circuit, hashIn_bin, nonce)
        if QC_switch == 1: 
            [qstate_bin, comp_time] = exp_quantum_operation(quantum_circuit, hashIn_bin, nonce, qpu_backend) #qstate is a 256 binary number

        print('qblock-output:', str(qstate_bin), 'length:', len(qstate_bin))

        xor_bin = xor(str(hashIn_bin), str(qstate_bin), 256)
        hashOut = hashlib.sha3_256(xor_bin.encode("ascii")).hexdigest()

        print('XOR:', str(xor_bin), 'length:', len(xor_bin))
        with open(filename_log, 'a') as o:
            o.write('XOR:'+ str(xor_bin) + 'length:'+ str(len(xor_bin)) + '\n')

        print('hashOut-hex:', str(hashOut), 'length:', len(hashOut), '\n\n')

        return [hashOut, comp_time]

    #mining based on multiple qPoW cycles, the function runs untill it finds the nonce satisfying the difficulty conditions
    def mine(block_number, transactions, previous_hash, prefix_zeros, QC_switch):

        prefix_str = '0'*prefix_zeros #sets the difficulty, in hex format, bin: multiply by 4
        
        start = time.time()
        comp_time_block = 0
        
        for i in range(MAX_NONCE):
#             random.seed(i)
            nonce = random.randint(0, MAX_NONCE)
            text = str(block_number) + transactions + previous_hash + str(nonce) #hash input
            print ('text:', text, '\n'  )
            [new_hash, comp_time] = qPoW(text,QC_switch, quantum_circuit, nonce)

            #write to log file
            with open(filename_log, 'a') as o:
                o.write(str(i) + ' nonce:' + str(nonce)+'\n')
                o.write('hashOut:'+ str(new_hash)+'\n\n')
            comp_time_block+=comp_time
            
            if new_hash.startswith(prefix_str):
                break

        print(f"qPoW completed. Successfully mined a block with a nonce value:{nonce}")
        with open(filename_log, 'a') as o:
            o.write('qPoW completed. Successfully mined a block with a nonce value' +str(nonce)+'\n')
        with open(filename_results, 'a') as o:
            o.write('qPoW completed.' +str(nonce)+'\n')
        with open(filename_bc, 'a') as o:
            o.write('computational time for mining a block:' +str(comp_time_block)+'\n')
        
        total_time = str((time.time() - start))
        print(f"mining ended. mining time: {total_time} seconds")
        print('final hash:', new_hash)
        print('suitable nonce:', nonce)
        
        return [new_hash, nonce, comp_time]

        raise BaseException(f"Couldn't find correct has after trying {MAX_NONCE} times")

    ### Quantum operations

    # converting hashIn_bin to a bit string to pass thru a quantum processor
    def break_up_4bit_values(hashIn_bin):

        array_4_bit_values = []
        i = 0

        while i < 64 : 
          four_bits = hashIn_bin[2+4*i:2+4*i+4]
          array_4_bit_values.append(four_bits)
          i = i + 1
        print("hashIn binary split into 4bit bins:", array_4_bit_values)
        return array_4_bit_values
    
    # setting the quantum circuit:
    def quantum_circuit(q_par, n_qreg):
        qreg_q = QuantumRegister(n_qreg, 'q')
        creg_c = ClassicalRegister(n_qreg, 'c')
        circuit = QuantumCircuit(qreg_q, creg_c)

        #repetitive circuit
#         for i in range(circ_layer):
            
        #n-qubit circuit  
        k = 0 #counter for the parameter values
        for i in range(n_qreg):   
            circuit.rx(q_par[k+i]*pi/8, qreg_q[i])
        k+=n_qreg
        
        for i in range(n_qreg):    
            circuit.rz(q_par[k+i]*pi/8, qreg_q[i])
        k+=n_qreg
        
        for i in range(n_qreg):
            for j in range(n_qreg):
                if j != i:
                    circuit.crx(q_par[k+j]*pi/8, qreg_q[n_qreg-1-i], qreg_q[n_qreg-1-j])
                else:
                    k-=1
            k+=n_qreg-1
        
        for i in range(n_qreg):   
            circuit.rx(q_par[k+i]*pi/8, qreg_q[i])
        k+=n_qreg
        
        for i in range(n_qreg):    
            circuit.rz(q_par[k+i]*pi/8, qreg_q[i])

        #measurements of all qubits
        for i in range(len(qreg_q)):
            circuit.measure(qreg_q[i], creg_c[i])
        
        return circuit
        
        
    # real(experimental) quantum computor run
    def exp_quantum_operation(quantum_circuit, hashIn, nonce, qpu_backend):

        #input hashIn string
        fourbit_array = break_up_4bit_values(hashIn)
        q_par = [int(fourbit_array[i],2) for i in range(len(fourbit_array)-1)] #throwing away the last string element
        circuit = quantum_circuit(q_par, n_qreg)
        
        #for IBMQ machines the circuit needs to go thru the transpilation (expression in basic gates)
        transpiled_circuit = transpile(circuit, qpu_backend, seed_transpiler=13)

        qpu_job = qpu_backend.run(transpiled_circuit, shots=20000)
        job_id = qpu_job.job_id()
        print("Job id", job_id)

        # Monitor job progress and wait until complete:
        job_monitor(qpu_job)

        # Get the job results (this method also waits for the Job to complete):
        results = qpu_job.result()
        print(results)
        
        with open(filename_log, 'a') as o:
            o.write(str(results) + '\n')

        with open(filename_results, 'a') as o:
            o.write("time taken: {} sec".format(results.time_taken) + '\n')
        
        comp_time = results.time_taken
        
        counts = {format(n, "03b"): 0 for n in range(8)}
        counts.update(results.get_counts(circuit))
        print(counts)

        # plot histograms
        hist_filename = filepath + 'exp_histogram' + job_id
        plot_histogram(counts, filename = hist_filename, figsize=(7, 5))

        #draw the quantum circuit
        drawC_filename = filepath + 'exp_q_circuit' + job_id
        drawTC_filename = filepath + 'exp_q_Tcircuit' + job_id
        circuit.draw('mpl', fold=0, filename = drawC_filename)
        transpiled_circuit.draw('mpl',fold=0, filename = drawTC_filename)

        #picking up the maximally probable state
        max_state = max(counts, key=counts.get)
        max_state256 = max_state

        for i in range(256 - len(max_state)):
            max_state256+='0' 

        with open(filename_results, 'a') as o:
            o.write(str(job_id) + ',' + str(max_state)+',')

        return [max_state256, comp_time] #4bit vector

    # quantum simulator run
    def sim_quantum_operation(quantum_circuit, hashIn, nonce):

        #input hashIn string
        fourbit_array = break_up_4bit_values(hashIn)
        q_par = [int(fourbit_array[i],2) for i in range(len(fourbit_array)-1)] #throwing away the last string element
        circuit = quantum_circuit(q_par, n_qreg)

        backend = BasicAer.get_backend(simulator_name) # run on local simulator by default 

        job = execute(circuit, backend, shots=20000)

        # Monitor job progress and wait until complete:
        job_monitor(job)

        # Get the job results (this method also waits for the Job to complete):
        results = job.result()
        print(results)

        with open(filename_log, 'a') as o:
            o.write(str(results) + '\n')

        with open(filename_results, 'a') as o:
            o.write("time taken: {} sec".format(results.time_taken) + '\n')
        
        comp_time = results.time_taken
        counts = results.get_counts(circuit)

        #plot histograms
        hist_filename = filepath + 'sim_histogram' + str(nonce)
        plot_histogram(counts, filename = hist_filename, figsize=(7, 7))

        #draw the quantum circuit
        drawC_filename = filepath + 'sim_q_circuit' + str(nonce)
        circuit.draw('mpl', fold=0, filename = drawC_filename)

        #picking up the maximally probable state
        max_state = max(counts, key=counts.get)
        print ('max_state:', max_state)
        max_state256 = max_state

        for i in range(256 - len(max_state)):
            max_state256+='0' 

        with open(filename_results, 'a') as o:
            o.write('max_state = '+str(max_state)+', '+'nonce = '+str(nonce)+', ' + '\n')


        return [max_state256, comp_time] #4bit vector

    def xor(a, b, n): #a,b - strings, n - length of the XOR output
        ans = ""

        # Loop to iterate over the
        # Binary Strings
        for i in range(n):

            # If the Character matches
            if (a[i] == b[i]):
                ans += "0"
            else:
                ans += "1"
        return ans


    ### Execute QPoW
    [new_hash, nonce, comp_time] = mine(block_number,transactions, previous_hash, difficulty, QC_switch)
    
    ### Verify with the quantum simulator

    def verify(nonce, block_number, transactions, previous_hash, prefix_zeros, quantum_circuit, QC_switch):
        prefix_str = '0'*prefix_zeros

        print('nonce:', nonce)
        text = str(block_number) + transactions + previous_hash + str(nonce)
        print('text:', text)
        [new_hash, ver_time] = qPoW(text, QC_switch, quantum_circuit, nonce)

        if new_hash.startswith(prefix_str):
            print(f"Verified: nonce {nonce}\n") 
            with open(filename_results, 'a') as o:
                o.write('Verified' + '\n')
            with open(filename_log, 'a') as o:
                o.write('Verified' + '\n')
            return [True, new_hash, ver_time]
        else:
            print(f"False:{nonce}\n")
            with open(filename_results, 'a') as o:
                o.write('False' + '\n')
            with open(filename_log, 'a') as o:
                o.write('False' + '\n')
            return [False, previous_hash, ver_time]
    
    [check, new_hash, ver_time] = verify(nonce, block_number, transactions, previous_hash, difficulty, quantum_circuit, QC_switch = 0)
    
    if check == True: 
        previous_hash = new_hash
        block_number+= 1
        counter+= 1 
        with open(filename_bc, 'a') as o:
            o.write('block_number:' + str(block_number-1) + '\n')
            o.write('nonce:' + str(nonce)+ '\n')
            o.write('previous_hash:' + str(previous_hash)+ '\n')
            o.write('transactions:' + str(transactions) +'\n')
            o.write('errors:' + str(errors) +'\n')
            o.write('time for verifying a block:' +str(ver_time)+'\n\n')
        
    else:
        previous_hash = previous_hash
        block_number = block_number
        errors += 1
        print('number of errors:', errors)
        